In [1]:
%pip install pdfplumber

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 25.1.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import pdfplumber



In [3]:
pdf_path = "JBCAP.pdf"

In [4]:
# Read tables from a PDF file
pf = pdfplumber.open(pdf_path)

In [5]:
pgs = pf.pages
data = []

In [6]:
for pg in range(79,140):
    z = pf.pages[pg].extract_table(table_settings={"vertical_strategy": "text"})
    if z:
        data.extend(z)

In [7]:
print(z)

[['', 'Sale Details', '', '', '', '', 'Purchase Deta', 'ils', '', '', '', 'G', 'ain / Loss', ''], ['Sell Date', 'Quantity S', 'ell Rate', 'Total Sale\nValue', 'Purchase\nDate', 'Purchase Stamp\nRate Duty', 'Actual Cost', 'FMV as on\n31-01-2018/\nIndexed Rate', 'Applicable\nE\nRate', 'ffective Cost', 'Days\nHeld', 'Short Term', 'Long Term', 'Effective LT'], ['21-Aug-2024', '325', '968.381', '3,14,723.71\n16,17,507.17\n14,37,867.00\n9,58,578.00\n4,80,513.96\n4,80,513.96\n14,28,675.90\n9,70,146.33\n9,70,146.33\n9,70,146.33\n6,69,92,612.94', '07-Jun-2023', '92.59', '30,091.46', '', '92.59', '30,091.46', '441', '', '2,84,632.25', '2,84,632.25'], ['21-Aug-2024', '1,675', '965.676', None, '07-Jun-2023', '92.59', '1,55,086.74', '', '92.59', '1,55,086.74', '441', '', '14,62,420.43', '14,62,420.43'], ['22-Aug-2024', '1,500', '958.578', None, '07-Jun-2023', '92.59', '1,38,883.65', '', '92.59', '1,38,883.65', '442', '', '12,98,983.35', '12,98,983.35'], ['22-Aug-2024', '1,000', '958.578', None, '07

In [11]:
import pandas
import numpy as np
df = pandas.DataFrame(data)

In [13]:
import camelot

c:\Users\DELL\coe\.venv\Lib\site-packages\pypdf\_crypt_providers\_cryptography.py:32: CryptographyDeprecationWarning: ARC4 has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.ARC4 and will be removed from cryptography.hazmat.primitives.ciphers.algorithms in 48.0.0.
  from cryptography.hazmat.primitives.ciphers.algorithms import AES, ARC4


In [26]:
tables = camelot.read_pdf(pdf_path, pages="80-140",flavor='stream')

In [34]:
print(tables[26])

<Table shape=(37, 14)>


In [27]:
df = tables[0].df

In [28]:
df1 = tables[1].df

In [ ]:
df1 = tables[1].df

In [35]:
df_main = pandas.DataFrame()

for table in tables:
    df_main = pandas.concat([df_main, table.df], ignore_index=True)